
# Assignment week 5 (summative)

Use the Openweathermap Current Whether Data API to get the temperature of all cities in UK. The output shall be a dataframe with two columns (name of city, temperature). 

Hint 1: To get the name of all cities in UK, consider scraping the website https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Kingdom.

Hint 2: You might find this post https://stackoverflow.com/questions/62475160/extract-words-from-string-before-number-python useful.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import json
import re

url = "https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Kingdom"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

cities_table = soup.find("table", {"class": "wikitable"})

city_names = []
for row in cities_table.find_all("tr")[1:]:
    cells = row.find_all("td")
    if len(cells) >= 2:
        city_name = cells[1].find("a").get("title")
        city_name = re.sub(r'[^\x00-\x7F]+', '', city_name)
        city_names.append(city_name)

city_temperatures = []

with open("APItemplate.json", "r") as file:
    API = json.load(file)
api_key = API["OpenWeather"]

for city in city_names:
    try:
        query = "q=" + city
        res = requests.get(
            f"http://api.openweathermap.org/data/2.5/weather?{query}&APPID={api_key}"
        )
        data = res.json()

        if 'main' in data and 'temp' in data['main']:
            temperature = data['main']['temp'] - 273.15
            city_temperatures.append((city, temperature))
        else:
            city_temperatures.append((city, "N/A"))
    except Exception as e:
        print(f"Error fetching data for {city}: {str(e)}")


df = pd.DataFrame(city_temperatures, columns=["Name of city", "Temperature"])
display(df)

,Name of city,Temperature
0,City of London,6.99
1,City of Westminster,7.0
2,Birmingham,16.33
3,City of Leeds,N/A
4,Glasgow,5.5
...,...,...
71,Ripon,6.72
72,Armagh,9.3
73,"Wells, Somerset",8.63
74,St Asaph,8.11
